# W3P2 - PART3 - yelp_foursquareEDA

assignment file part 2

In [1]:
from datetime import datetime
import pandas as pd
import json
import warnings
warnings.filterwarnings('ignore')

# Foursquare

In [2]:
# Import data from file
df = pd.read_csv('../data/df_ctybks_toronto.csv')
df.shape

df_yelp = pd.read_csv('../data/yelpPOI/df_yelp_sun1436h.csv')
# Save working copies:
dfy = df_yelp.copy()

df_fsq = pd.read_csv('../data/fsqPOI/df_fsq_sun1400h.csv')
# Save working copies:
dff = df_fsq.copy()

In [3]:
df_fsq.shape

(32709, 15)

In [4]:
df_fsq.head(1)

,lat-long,fsq_id,cat_id,category_name,categories,name,distance,latitude,longitude,address,popularity,open_now,rating,rating_count,my_timestamp
0,"43.665269,-79.319796",4deb8ba688774880e3387c0c,17065.0,Farmers' Market,"[{'id': 17065, 'name': ""Farmers' Market"", 'ico...",Leslieville Farmers Market,40,43.664679,-79.319687,NaN,0.86858,False,8.6,NaN,2022-12-04 12:07:13.609094


## Cleaning and Normalizing the nested table values:

The function which created the DataFrame made sure that each POI had at least one category name and ID, but we can see above that many have two or even three which can be normalized to flatten the hierarchy.

In [5]:
# Normalize the categories:
df_fsqcat = pd.json_normalize(df_fsq['categories'][0])
df_fsqcat.head()

NotImplementedError: 

In [ ]:
# normalize the resulting 3 columns into 3 tables in the example below:
df_fsqcat0 = pd.json_normalize(df_fsqcat[0])
df_fsqcat1 = pd.json_normalize(df_fsqcat[1])
df_fsqcat2 = pd.json_normalize(df_fsqcat[2])
df_fsqcat0.head()

In [ ]:
# Add new category labels back onto the df: 
df_fsq['cat1_id'] = (df_fsqcat0['id'].astype('Int64'))
df_fsq['cat1_name'] = df_fsqcat0['name']
df_fsq['cat1_icon'] = df_fsqcat0['icon.prefix']
df_fsq['cat1_icon.suffix'] = df_fsqcat0['icon.suffix']

df_fsq['cat2_id'] = df_fsqcat1['id'].astype('Int64')
df_fsq['cat2_name'] = df_fsqcat1['name']
df_fsq['cat2_icon'] = df_fsqcat1['icon.prefix']
df_fsq['cat2_icon.suffix'] = df_fsqcat1['icon.suffix']

df_fsq['cat3_id'] = df_fsqcat2['id'].astype('Int64')
df_fsq['cat3_name'] = df_fsqcat2['name']
df_fsq['cat3_icon'] = df_fsqcat2['icon.prefix']
df_fsq['cat3_icon.suffix'] = df_fsqcat2['icon.suffix']

# Remove the now redundant 'category' and 'name' columns:
df_fsq = df_fsq.drop(['categories', 'category_name', 'cat_id', ], axis = 1)

df_fsq.head(1)

### FOURSQUARE POI:

In [ ]:
# Clone a working copy:
dff = df_fsq.copy()

# Drop columns from our copy df not currently needed:
dff = dff.drop(['fsq_id', 'cat1_icon', 'cat2_icon', 'cat3_icon', 'cat1_icon.suffix', 'cat2_icon.suffix', 'cat3_icon.suffix', 'address'], axis = 1)
dff.head(2)

Convert the timestamp data to the Toronto's time zone:

In [ ]:
from datetime import datetime
import pytz

In [ ]:
# create both timezone objects
old_timezone = pytz.timezone("US/Mountain")
new_timezone = pytz.timezone("US/Eastern")

# Confirm current timezone
print(dff['my_timestamp'].iloc[0])

# two-step process
for i in range(dff.shape[0]):
  localized_timestamp = old_timezone.localize(dff['my_timestamp'].iloc[i])
  dff['my_timestamp'].iloc[i] = localized_timestamp.astimezone(new_timezone)
  dff['my_timestamp'].iloc[i] = localized_timestamp.astimezone(new_timezone)


# Has converted to new timezone:
print(dff['my_timestamp'].iloc[0])

Put your parsed results into a DataFrame

In [ ]:
dff.to_csv(f'../data/df_fsq_1400h.csv', index= False)

dff.head(5)

get rid of the time stamp and save in table name instead.


In [ ]:
dfDataRunTime = 

#get rid of the time stamp and save in table name instead.

#### SAVE IT!!

In [ ]:
#change the saving path and name for stats model df

df_fsq.to_csv(f'../data/df_fsq{dfDataRunTime}.csv', index=False)

----
# Yelp

In [ ]:
df_yelp.shape

## Cleaning and Normalizing the nested table values:

The function which created the DataFrame made sure that each POI had at least one category name and alias, but we can see above that many have two or even three which can be normalized to flatten the hierarchy.

In [ ]:
# Normalize the categories:
df_yelpcat = pd.json_normalize(df_yelp['categories'][0])
df_yelpcat.head()

In [ ]:
# normalize the resulting 3 columns into 3 tables in the example below:
df_yelpcat0 = pd.json_normalize(df_yelpcat[0])
df_yelpcat1 = pd.json_normalize(df_yelpcat[1])
df_yelpcat2 = pd.json_normalize(df_yelpcat[2])
df_yelpcat0.head()

In [ ]:
# Add new category labels back onto the df: 
df_yelp['cat1_alias'] = df_yelpcat0['alias']
df_yelp['cat1_title'] = df_yelpcat0['title']

df_yelp['cat2_alias'] = df_yelpcat1['alias']
df_yelp['cat2_title'] = df_yelpcat1['title']

df_yelp['cat3_alias'] = df_yelpcat2['alias']
df_yelp['cat3_title'] = df_yelpcat2['title']

# Remove the now redundant 'category' and 'name' columns:
df_yelp = df_yelp.drop(['categories', 'category_name'], axis = 1)

df_yelp.head(1)

### SAVE IT!!!

In [ ]:
df_yelp.to_csv(f'../data/yelpPOI/df_yelp{datetime.now()}.csv', index=False)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

### YELP POI:

In [ ]:
# Clone a working copy:
df = df_yelp.copy()

the addresses look chunky which diminishes the aesthetic of the table, and I expect this format might be useful for printing them in mailing format.  I won't be using it for my analysis at this point anyway, but I would certainly research this more before I made decisions about changing the format.  


In [ ]:
# Drop columns from our copy df not currently needed:
df = df.drop(['address', 'cat1_alias', 'cat2_alias', 'cat3_alias'], axis = 1)
df.head(2)

flip the "is_closed" column to remove the confusing double negative, and enable foursquare comparison:

In [ ]:
df['is_closed'] = df['is_closed'] == False
# Rename the heading to match:
df = df.rename(columns = {'is_closed':'is_open'})
df.head(2)

Convert the timestamp data to the Toronto's time zone:

In [ ]:
from datetime import datetime
import pytz

In [ ]:
# create both timezone objects
old_timezone = pytz.timezone("US/Mountain")
new_timezone = pytz.timezone("US/Eastern")

# Confirm current timezone
print(df['my_timestamp'].iloc[0])

# two-step process
for i in range(df.shape[0]):
  localized_timestamp = old_timezone.localize(df['my_timestamp'].iloc[i])
  df['my_timestamp'].iloc[i] = localized_timestamp.astimezone(new_timezone)
  df['my_timestamp'].iloc[i] = localized_timestamp.astimezone(new_timezone)


# Has converted to new timezone:
print(df['my_timestamp'].iloc[0])

In [ ]:
df['distance'] = round(df['distance'], 1)
df.head(2)

In [ ]:
test = df['my_timestamp'].iloc[0]
print(test)
print(datetime.date(test))
print(datetime.time(test))
print(len('2022-12-04 00:09'))

##### Categorical encoding: price


In [ ]:
print(df['price'].value_counts())
print(df['price'].isna().value_counts())
df['price'].unique()

In [ ]:
# order in array above will be same as the keys,
keys = df['price'].unique()
# Setting ordinal numerical values to match the order:
values = [2, 1, 3, None, 4]
price_map = dict(zip(keys, values))
price_map

In [ ]:
# Replace
df['price'] = df['price'].map(price_map)
# Validate counts unaffected:
print(df['price'].value_counts())

df.head(1)

## SAVE IT!!

In [ ]:
df.to_csv(f'../data/df_yelp_1440h.csv', index= False)

df.head(5)